# Word2Vec (Skipgram )

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
import nltk
from nltk.corpus import semcor, stopwords

nltk.download('semcor')
nltk.download('stopwords')

[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\surya\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\surya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'NVIDIA GeForce RTX 2080 with Max-Q Design')

In [4]:
semcor.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

## 1. Load data

In [5]:
#1. tokenization
corpus = semcor.sents()

In [6]:
#2. numeralization
#find unique words
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(corpus))) #all the words we have in the system - <UNK>

In [7]:
#create handy mapping between integer and word
word2index = {v:idx for idx, v in enumerate(vocabs)}


In [8]:
vocabs.append('<UNK>')
word2index['<UNK>'] = len(vocabs) - 1  

In [9]:
index2word = {v:k for k, v in word2index.items()}

## 2. Prepare train data

In [10]:
#create pairs of center word, and outside word

def random_batch(batch_size, corpus, window_size=2):

    skipgrams = []

    #loop each corpus
    for doc in corpus:
        #look from the 2nd word until second last word
        for i in range(window_size, len(doc)-window_size):
            #center word
            center = word2index[doc[i]]
            
            #outside words (window size on both sides of the center word)
            outside_start =  i - window_size
            outside_end =  i + window_size + 1 # +1 because the end index is exclusive

            # outside = []
            # loop through the outside words, append to the list 'outside'
            for j in range(outside_start, outside_end):
                if i != j:  # Skip the center word
                    outside= word2index[doc[j]]
                    skipgrams.append([center, outside])

                
    random_index = np.random.choice(range(len(skipgrams)), batch_size, replace=False)
    
    inputs, labels = [], []
    for index in random_index:
        inputs.append([skipgrams[index][0]])
        labels.append([skipgrams[index][1]])
        
    return np.array(inputs), np.array(labels)
            
x, y = random_batch(2, corpus)

In [11]:
x.shape  #batch_size, 1

(2, 1)

In [12]:
x

array([[33356],
       [19595]])

In [13]:
y.shape  #batch_size 1

(2, 1)

## 3. Model

$$J(\theta) = -\frac{1}{T}\sum_{t=1}^{T}\sum_{\substack{-m \leq j \leq m \\ j \neq 0}}\log P(w_{t+j} | w_t; \theta)$$

where $P(w_{t+j} | w_t; \theta) = $

$$P(o|c)=\frac{\exp(\mathbf{u_o^{\top}v_c})}{\sum_{w=1}^V\exp(\mathbf{u_w^{\top}v_c})}$$

where $o$ is the outside words and $c$ is the center word

$$P(o|c)=\frac{\exp(\mathbf{u_o^{\top}v_c})}{\sum_{w=1}^V\exp(\mathbf{u_w^{\top}v_c})}$$

In [14]:
class Skipgram(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(Skipgram, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
    
    def forward(self, center, outside, all_vocabs):
        center_embedding     = self.embedding_center(center)  #(batch_size, 1, emb_size)
        outside_embedding    = self.embedding_center(outside) #(batch_size, 1, emb_size)
        all_vocabs_embedding = self.embedding_center(all_vocabs) #(batch_size, voc_size, emb_size)
        
        top_term = torch.exp(outside_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2))
        #batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) = (batch_size, 1) 

        lower_term = all_vocabs_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2)
        #batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size, 1) = (batch_size, voc_size) 
        
        lower_term_sum = torch.sum(torch.exp(lower_term), 1)  #(batch_size, 1)
        
        loss = -torch.mean(torch.log(top_term / lower_term_sum))  #scalar
        
        return loss
        

In [15]:
#prepare all vocabs

batch_size = 2
voc_size   = len(vocabs)

def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)

all_vocabs = prepare_sequence(list(vocabs), word2index).expand(batch_size, voc_size)
all_vocabs

tensor([[    0,     1,     2,  ..., 43414, 43415, 43416],
        [    0,     1,     2,  ..., 43414, 43415, 43416]])

In [16]:
model = Skipgram(voc_size, 2)
model

Skipgram(
  (embedding_center): Embedding(43417, 2)
  (embedding_outside): Embedding(43417, 2)
)

In [17]:
input_tensor = torch.LongTensor(x)
label_tensor = torch.LongTensor(y)

In [18]:
loss = model(input_tensor, label_tensor, all_vocabs)

In [19]:
loss

tensor(10.8671, grad_fn=<NegBackward0>)

## 4. Training

In [20]:
batch_size = 2
emb_size   = 2
model      = Skipgram(voc_size, emb_size)
optimizer  = optim.Adam(model.parameters(), lr=0.001)

In [21]:
import time

def log_epoch(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [24]:
start = time.time()
num_epochs = 100
window_size = 5

for epoch in range(num_epochs):
    
    #get batch
    input_batch, label_batch = random_batch(batch_size, corpus, window_size)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)
    
    #predict
    loss = model(input_tensor, label_tensor, all_vocabs)
    
    #backprogate
    optimizer.zero_grad()
    loss.backward()
    
    #update alpha
    optimizer.step()
    # Log epoch time
    epoch_mins, epoch_secs = log_epoch(start, time.time())
    
    #print the loss
    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch + 1} | Loss: {loss:.6f} | Time: {epoch_mins}m {epoch_secs}s")

Epoch: 10 | Loss: 10.053512 | Time: 2m 48s
Epoch: 20 | Loss: 12.454390 | Time: 5m 37s
Epoch: 30 | Loss: 10.791930 | Time: 8m 27s
Epoch: 40 | Loss: 11.176625 | Time: 11m 19s
Epoch: 50 | Loss: 11.075615 | Time: 14m 10s
Epoch: 60 | Loss: 12.948166 | Time: 17m 1s
Epoch: 70 | Loss: 11.798746 | Time: 19m 51s
Epoch: 80 | Loss: 11.828511 | Time: 22m 46s
Epoch: 90 | Loss: 12.305994 | Time: 25m 40s
Epoch: 100 | Loss: 12.651293 | Time: 28m 31s


## 5. Plot the embeddings

Is fruit really near to banana?
Is fruit really far from cat?

In [25]:
vocabs

['',
 'Harold',
 'tongs',
 'wanted',
 'negatively',
 'anchored',
 'Miss',
 'completing',
 'buoys',
 'receptionist',
 'imaging',
 'Drum',
 'Doctors',
 'enlightened',
 'hideaway',
 'blandly',
 'imperative',
 'lettin',
 'creek-filled',
 'Payments',
 'fawn',
 'Corder',
 'canister',
 'crude',
 'Aggie',
 'Entwhistle',
 'pale',
 'unlined',
 'pestering',
 'tabula',
 'Depression',
 'f',
 'tobacco',
 'Schelling',
 'haphazard',
 'dived',
 'Course',
 'awful',
 'Manderscheid',
 'ornamented',
 '920',
 'creep',
 'Manufacturers',
 '8293',
 'scopes',
 'out-moded',
 'deposits',
 'lettering',
 'posture',
 'Result',
 'Trackdown',
 'equal',
 'Culture',
 'servitors',
 'Carmer',
 'armor',
 'banners',
 'consolidation',
 'rattler',
 'Waggin',
 '2.1',
 'Bester',
 'calmed',
 'Brevard',
 'peasant',
 'exuberant',
 'Isolde',
 'falsehood',
 'dispute',
 'medical',
 'encounter',
 'stimulus',
 'steepest',
 'virtuosi',
 'tunneled',
 'nudes',
 'Korra',
 'Sid',
 'trades',
 'Multiply',
 'no-hit',
 'speak-easy',
 'thanks',


In [26]:
banana = torch.LongTensor([word2index['banana']])
banana

tensor([33519])

In [27]:
banana_embed_c = model.embedding_center(banana)
banana_embed_o = model.embedding_outside(banana)
banana_embed   = (banana_embed_c + banana_embed_o) / 2
banana_embed

tensor([[-0.9419, -0.0039]], grad_fn=<DivBackward0>)

In [28]:
banana_embed_o

tensor([[-1.1480,  0.6964]], grad_fn=<EmbeddingBackward0>)

In [29]:
def get_embed(word):
    try:
        index = word2index[word]
    except:
        index = word2index['<UNK>']
        
    word = torch.LongTensor([word2index[word]])
    
    embed_c = model.embedding_center(word)
    embed_o = model.embedding_outside(word)
    embed   = (embed_c + embed_o) / 2
    
    return embed[0][0].item(), embed[0][1].item()

In [30]:
get_embed('fruit')

(0.4439716339111328, 0.6431342959403992)

In [31]:
get_embed('cat')

(-0.344612717628479, -0.31181710958480835)

In [32]:
get_embed('dog')

(0.7699832320213318, -0.04242171347141266)

In [33]:
get_embed('banana')

(-0.941861093044281, -0.003913760185241699)

## 6. Cosine similarity

In [34]:
banana = get_embed('banana')
banana

(-0.941861093044281, -0.003913760185241699)

In [35]:
fruit = get_embed('fruit')
fruit

(0.4439716339111328, 0.6431342959403992)

In [36]:
unk = get_embed('<UNK>')
unk

(0.053452007472515106, 0.762205183506012)

In [37]:
np.array(banana) @ np.array(unk)

-0.05332745448366483

In [38]:
#more formally is to divide by its norm
def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

print(cosine_similarity(np.array(banana), np.array(unk)))
print(cosine_similarity(np.array(banana), np.array(fruit)))

-0.07410082563838405
-0.5715211478067245


## Export Model

In [39]:
import pickle

In [40]:
filename = 'skipgram_model.pkl'
pickle.dump(model, open(filename, 'wb'))